In [ ]:
import pandas as pd
import sqlite3
import logging
from ingestion_db import ingest_db

logging.basicConfig(
    filename="logs/get_vendor_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="w"
)
        
def create_vendor_summary(conn):
    '''this function will merge the different tables to get the overall vendor summary and adding new columns in the resultant data'''
   
    vendor_sales_summary = pd.read_sql_query("""
    WITH FreightSummary as (
        SELECT VendorNumber, SUM(Freight) as FreightCost FROM vendor_invoice GROUP BY VendorNumber
),

PurchaseSummary as (
    SELECT
        p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice,
        pp.Price as ActualPrice, pp.Volume,
        SUM(p.Quantity) as TotalPurchaseQuantity,
        SUM(p.Dollars) as TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp ON p.Brand = pp.Brand 
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),

SalesSummary as (
    SELECT
        VendorNo, Brand,
        SUM(SalesQuantity) as TotalSalesQuantity,
        SUM(SalesPrice) as TotalSalesPrice,
        SUM(SalesDollars) as TotalSalesDollars,
        SUM(ExciseTax) as TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)

    SELECT
        ps.VendorNumber, ps.VendorName, ps.Brand, ps.Description, ps.PurchasePrice,
        ps.ActualPrice, ps.Volume, ps.TotalPurchaseQuantity, ps.TotalPurchaseDollars,
        ss.TotalSalesQuantity, ss.TotalSalesDollars, ss.TotalSalesPrice, ss.TotalExciseTax,
        fs.FreightCost
    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
    LEFT JOIN FreightSummary fs ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.TotalPurchaseDollars DESC
    """, conn)

    return vendor_sales_summary


def clean_data(df):
    '''this function will clean the data and create new aggregate columns'''

    # The Cleaning Steps
    # a. Changing datatype to float
    df['Volume'] =  df['Volume'].astype('float')

    # b. Filling missing value with 0
    df.fillna(0,inplace = True)

    # c. Removing spaces from categorical columns 
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # d. Creating new columns for better analysis
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
    df['ProfitMargin'] = df['GrossProfit'] / df['TotalSalesDollars'] * 100
    df['QuantityTurnover'] = df['TotalSalesQuantity'] / df['TotalPurchaseQuantity']
    df['SalestoPurchaseRatio'] = df['TotalSalesDollars'] / df['TotalPurchaseDollars']
    
    return df

if __name__ == '__main__':
    #creating database connection
    conn = sqlite3.connect('inventory.db')

    # ('Creating Vendor summary table') which connects to get_summary_log.db
    logging.info('Starting vendor summary process.')
    
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info('Cleaning Data and calculating aggregates.....')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())
    
    # This call saves the data to the database
    logging.info('Ingesting Data to database (replacing table).....')
    ingest_db(clean_df, 'vendor_sales_summary', conn)
    
    logging.info('Completed process successfully')